### 0. Data

In [ ]:
!pip install pycaret

In [ ]:
import numpy as np
import pandas as pd
from pycaret.classification import *
from sklearn.datasets import make_classification
from sklearn.preprocessing import OrdinalEncoder, StandardScaler

In [ ]:
loan_data_backup = pd.read_csv('/content/drive/MyDrive/fin_credit/loan_data_inputs_train.csv')

In [ ]:
loan_data = loan_data_backup.copy()

### 1. Preprocessing

In [ ]:
loan_data['emp_length_int'] = loan_data['emp_length'].str.replace('\+ years', '',regex=True)
loan_data['emp_length_int'] = loan_data['emp_length_int'].str.replace('< 1 year', str(0))
loan_data['emp_length_int'] = loan_data['emp_length_int'].fillna('0')
loan_data['emp_length_int'] = loan_data['emp_length_int'].str.replace(' years', '')
loan_data['emp_length_int'] = loan_data['emp_length_int'].str.replace(' year', '')

In [ ]:
loan_data['emp_length_int'] = pd.to_numeric(loan_data['emp_length_int'])


In [ ]:
loan_data['earliest_cr_line_date'] = pd.to_datetime(loan_data['earliest_cr_line'], format = '%b-%y')

In [ ]:
loan_data['mths_since_earliest_cr_line'] = (
    (pd.to_datetime('2017-12-01') - loan_data['earliest_cr_line_date']).dt.days / 30.44
).round()

In [ ]:
loan_data['mths_since_earliest_cr_line'][loan_data['mths_since_earliest_cr_line'] < 0] = loan_data['mths_since_earliest_cr_line'].max()

In [ ]:
loan_data['term_int'] = loan_data['term'].str.replace(' months', '')

In [ ]:
loan_data['term_int'] = pd.to_numeric(loan_data['term'].str.replace(' months', ''))

In [ ]:
loan_data['issue_d_date'] = pd.to_datetime(loan_data['issue_d'], format = '%b-%y')


loan_data['mths_since_issue_d'] = (
    (pd.to_datetime('2017-12-01') - loan_data['issue_d_date']).dt.days / 30.44
).round()

In [ ]:
loan_data['total_rev_hi_lim'].fillna(loan_data['funded_amnt'], inplace=True)

In [ ]:
loan_data['annual_inc'].fillna(loan_data['annual_inc'].mean(), inplace=True)

In [ ]:
loan_data['mths_since_earliest_cr_line'].fillna(0, inplace=True)
loan_data['acc_now_delinq'].fillna(0, inplace=True)
loan_data['total_acc'].fillna(0, inplace=True)
loan_data['pub_rec'].fillna(0, inplace=True)
loan_data['open_acc'].fillna(0, inplace=True)
loan_data['inq_last_6mths'].fillna(0, inplace=True)
loan_data['delinq_2yrs'].fillna(0, inplace=True)
loan_data['emp_length_int'].fillna(0, inplace=True)

In [ ]:
loan_data['y'] = np.where(loan_data['loan_status'].isin(['Charged Off', 'Default',
                                                       'Does not meet the credit policy. Status:Charged Off',
                                                       'Late (31-120 days)']), 0, 1)

In [ ]:
pd.options.display.max_columns = None
loan_data.head()

,Unnamed: 0,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,emp_length_int,earliest_cr_line_date,mths_since_earliest_cr_line,term_int,issue_d_date,mths_since_issue_d,grade:A,grade:B,grade:C,grade:D,grade:E,grade:F,grade:G,home_ownership:ANY,home_ownership:MORTGAGE,home_ownership:NONE,home_ownership:OTHER,home_ownership:OWN,home_ownership:RENT,verification_status:Not Verified,verification_status:Source Verified,verification_status:Verified,loan_status:Charged Off,loan_status:Current,loan_status:Default,loan_status:Does not meet the credit policy. Status:Charged Off,loan_status:Does not meet the credit policy. Status:Fully Paid,loan_status:Fully Paid,loan_status:In Grace Period,loan_status:Late (16-30 days),loan_status:Late (31-120 days),purpose:car,purpose:credit_card,purpose:debt_consolidation,purpose:educational,purpose:home_improvement,purpose:house,purpose:major_purchase,purpose:medical,purpose:moving,purpose:other,purpose:renewable_energy,purpose:small_business,purpose:vacation,purpose:wedding,addr_state:AK,addr_state:AL,addr_state:AR,addr_state:AZ,addr_state:CA,addr_state:CO,addr_state:CT,addr_state:DC,addr_state:DE,addr_state:FL,addr_state:GA,addr_state:HI,addr_state:IA,addr_state:ID,addr_state:IL,addr_state:IN,addr_state:KS,addr_state:KY,addr_state:LA,addr_state:MA,addr_state:MD,addr_state:ME,addr_state:MI,addr_state:MN,addr_state:MO,addr_state:MS,addr_state:MT,addr_state:NC,addr_state:NE,addr_state:NH,addr_state:NJ,addr_state:NM,addr_state:NV,addr_state:NY,addr_state:OH,addr_state:OK,addr_state:OR,addr_state:PA,addr_state:RI,addr_state:SC,addr_state:SD,addr_state:TN,addr_state:TX,addr_state:UT,addr_state:VA,addr_state:VT,addr_state:WA,addr_state:WI,addr_state:WV,addr_state:WY,initial_list_status:f,initial_list_status:w,y
0,0,1077501,1296599,5000,5000,4975.0,36 months,10.65,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-11,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/22/11 > I need to upgra...,credit_card,Computer,860xx,AZ,27.65,0.0,Jan-85,1.0,NaN,NaN,3.0,0.0,13648,83.7,9.0,f,0.0,0.0,5861.071414,5831.78,5000.00,861.07,0.00,0.00,0.00,Jan-15,171.62,NaN,Jan-16,0.0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5000.0,NaN,NaN,NaN,10,1985-01-01,395.0,36,2011-12-01,72.0,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,1
1,1,1077430,1314167,2500,2500,2500.0,60 months,15.27,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-11,Charged Off,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/22/11 > I plan to use t...,car,bike,309xx,GA,1.00,0.0,Apr-99,5.0,

In [ ]:
df = loan_data.loc[: , ['grade',
'home_ownership',
'addr_state',
'verification_status',
'purpose',
'initial_list_status',
'term_int',
'emp_length_int',
'mths_since_issue_d',
'int_rate',
'mths_since_earliest_cr_line',
'delinq_2yrs',
'inq_last_6mths',
'open_acc',
'pub_rec',
'total_acc',
'acc_now_delinq',
'total_rev_hi_lim',
'annual_inc',
'dti',
#'mths_since_last_delinq',
'y'
]]

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 466285 entries, 0 to 466284
Data columns (total 21 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   grade                        466285 non-null  object 
 1   home_ownership               466285 non-null  object 
 2   addr_state                   466285 non-null  object 
 3   verification_status          466285 non-null  object 
 4   purpose                      466285 non-null  object 
 5   initial_list_status          466285 non-null  object 
 6   term_int                     466285 non-null  int64  
 7   emp_length_int               466285 non-null  int64  
 8   mths_since_issue_d           466285 non-null  float64
 9   int_rate                     466285 non-null  float64
 10  mths_since_earliest_cr_line  466285 non-null  float64
 11  delinq_2yrs                  466285 non-null  float64
 12  inq_last_6mths               466285 non-null  float64
 13 

In [ ]:
df.head()

,grade,home_ownership,addr_state,verification_status,purpose,initial_list_status,term_int,emp_length_int,mths_since_issue_d,int_rate,mths_since_earliest_cr_line,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,total_acc,acc_now_delinq,total_rev_hi_lim,annual_inc,dti,y
0,B,RENT,AZ,Verified,credit_card,f,36,10,72.0,10.65,395.0,0.0,1.0,3.0,0.0,9.0,0.0,5000.0,24000.0,27.65,1
1,C,RENT,GA,Source Verified,car,f,60,0,72.0,15.27,224.0,0.0,5.0,3.0,0.0,4.0,0.0,2500.0,30000.0,1.00,0
2,C,RENT,IL,Not Verified,small_business,f,36,10,72.0,15.96,193.0,0.0,2.0,2.0,0.0,10.0,0.0,2400.0,12252.0,8.72,1
3,C,RENT,CA,Source Verified,other,f,36,10,72.0,13.49,262.0,0.0,1.0,10.0,0.0,37.0,0.0,10000.0,49200.0,20.00,1
4,B,RENT,OR,Source Verified,other,f,60,1,72.0,12.69,263.0,0.0,0.0,15.0,0.0,38.0,0.0,3000.0,80000.0,17.94,1


## 2. AutoML

In [ ]:
# PyCaret 환경 설정
clf_setup = setup(data=df, target='y', session_id=42)

best_model = compare_models(fold=5)

print(best_model)

#save_model(best_model, 'best_autoML_model')

predictions = predict_model(best_model, data=df)
print(predictions.head())

,Description,Value
0,Session id,42
1,Target,y
2,Target type,Binary
3,Original data shape,"(466285, 21)"
4,Transformed data shape,"(466285, 46)"
5,Transformed train set shape,"(326399, 46)"
6,Transformed test set shape,"(139886, 46)"
7,Numeric features,14
8,Categorical features,6
9,Preprocess,True


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
ridge,Ridge Classifier,0.8907,0.6920,1.0000,0.8907,0.9422,0.0001,0.0033,2.8100
ada,Ada Boost Classifier,0.8907,0.6950,0.9999,0.8908,0.9422,0.0012,0.0143,20.4220
gbc,Gradient Boosting Classifier,0.8907,0.7007,0.9999,0.8908,0.9422,0.0016,0.0196,81.0580
lightgbm,Light Gradient Boosting Machine,0.8907,0.7013,0.9999,0.8908,0.9422,0.0017,0.0191,15.4760
dummy,Dummy Classifier,0.8907,0.5000,1.0000,0.8907,0.9422,0.0000,0.0000,2.4120
rf,Random Forest Classifier,0.8906,0.6750,0.9997,0.8909,0.9421,0.0029,0.0233,73.5840
lr,Logistic Regression,0.8905,0.6115,0.9997,0.8908,0.9421,0.0012,0.0099,37.3300
lda,Linear Discriminant Analysis,0.8904,0.6920,0.9991,0.8911,0.9420,0.0074,0.0351,4.2740
xgboost,Extreme Gradient Boosting,0.8903,0.6941,0.9986,0.8913,0.9419,0.0110,0.0429,7.4680
et,Extra Trees Classifier,0.8901,0.6619,0.9987,0.8911,0.9418,0.0068,0.0293,68.3100


Processing:   0%|          | 0/65 [00:00<?, ?it/s]

RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
                max_iter=None, positive=False, random_state=42, solver='auto',
                tol=0.0001)
Transformation Pipeline and Model Successfully Saved


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Ridge Classifier,0.8907,0.5000,1.0000,0.8907,0.9422,0.0001,0.0031


  grade home_ownership addr_state verification_status         purpose  \
0     B           RENT         AZ            Verified     credit_card   
1     C           RENT         GA     Source Verified             car   
2     C           RENT         IL        Not Verified  small_business   
3     C           RENT         CA     Source Verified           other   
4     B           RENT         OR     Source Verified           other   

  initial_list_status  term_int  emp_length_int  mths_since_issue_d  int_rate  \
0                   f        36              10                72.0     10.65   
1                   f        60               0                72.0     15.27   
2                   f        36              10                72.0     15.96   
3                   f        36              10                72.0     13.49   
4                   f        60               1                72.0     12.69   

   ...  inq_last_6mths  open_acc  pub_rec  total_acc  acc_now_delinq  \
0 

In [ ]:
# 적용된 전처리 정보 출력
from pycaret.classification import get_config
preprocessing_pipeline = get_config('pipeline')
print(preprocessing_pipeline)

Pipeline(memory=FastMemory(location=/tmp/joblib),
         steps=[('numerical_imputer',
                 TransformerWrapper(exclude=None,
                                    include=['term_int', 'emp_length_int',
                                             'mths_since_issue_d', 'int_rate',
                                             'mths_since_earliest_cr_line',
                                             'delinq_2yrs', 'inq_last_6mths',
                                             'open_acc', 'pub_rec', 'total_acc',
                                             'acc_now_delinq',
                                             'total_rev_hi_lim', 'annual_inc',
                                             'dti'],
                                    transformer=SimpleImpu...
                                                              handle_unknown='value',
                                                              return_df=True,
                                                    